### create resources

In [1]:
import time

from azure.identity import AzureCliCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient
from azure.mgmt.resource import ResourceManagementClient

In [2]:
subscription_id  = '<your-subscription-id>'
region = 'eastus'

project = 'testopenai'
region = 'eastus'

environment = '-dev-'
resource_group_name = 'rg-' + project + environment + region + '-001' # create names using convention
openai_name = 'oai-' + project + environment + region + '-001'
deployment_name = 'gpt-' + project + environment + region + '-001'

In [3]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, subscription_id) # instanciate client object
cognitive_client = CognitiveServicesManagementClient(credential=AzureCliCredential(), subscription_id=subscription_id)

Create Resource Group

In [4]:
resource_group = resource_client.resource_groups.create_or_update(resource_group_name, {'location': region})
print(f'Provisioned {resource_group.type} named {resource_group.name} in {resource_group.location}')

Provisioned Microsoft.Resources/resourceGroups named rg-testopenai-dev-eastus-001 in eastus


try:
    resource_group = resource_client.resource_groups.create_or_update(resource_group_name, {'location': region})
    print(f'Provisioned {resource_group.type} named {resource_group.name} in {resource_group.location}')
except:
    print('Not authenticated. Please authenticate first in the Azure CLI using az login and try again')

Create Cognitive Serivces Account

In [5]:
account_params = {'type' : 'Microsoft.CognitiveServices/accounts', 
                  'location': region, 
                  'sku': {'name': 'S0'}, 
                  'kind': 'OpenAI',
                  'tags' : {}}

In [11]:
cognitive_account = cognitive_client.accounts.begin_create(
                                                    resource_group_name = resource_group_name,
                                                    account_name = openai_name,
                                                    account = account_params)

In [12]:
for element in cognitive_client.accounts.list():
    print(element.name)

oai-testopenai-dev-eastus-001


In [13]:
cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

In [20]:
while cognitive_account.as_dict()['properties']['provisioning_state'] == 'Creating':
    print(cognitive_account.as_dict()['properties']['provisioning_state'], 'waiting 30 seconds....')
    time.sleep(30)
    cognitive_account = cognitive_client.accounts.get(resource_group_name = resource_group_name,
                                                  account_name = openai_name)

print(cognitive_account.as_dict()['properties']['provisioning_state'])

Creating waiting 10 seconds....
Succeeded


Deploy a model

In [16]:
deployment_properties = {'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo', 'version': '0301'},
                         'scale_settings': {'scale_type': 'Standard'},
                         'capabilities': {'completion': 'true', 'chatCompletion': 'true'}}

deployment_params = {'type' : 'Microsoft.CognitiveServices/accounts/deployments', 
                     'properties': deployment_properties}

In [17]:
deployment = cognitive_client.deployments.begin_create_or_update(resource_group_name = resource_group_name,
                                                                 account_name = openai_name,
                                                                 deployment_name = deployment_name,
                                                                 deployment= deployment_params)

In [18]:
for model in cognitive_client.deployments.list(resource_group_name = resource_group.name,
                                                                 account_name = openai_name):
    print(model.as_dict())

{'id': '/subscriptions/b1fcc415-9d99-40b4-9dc3-dea5e9775093/resourceGroups/rg-testopenai-dev-eastus-001/providers/Microsoft.CognitiveServices/accounts/oai-testopenai-dev-eastus-001/deployments/gpt-testopenai-dev-eastus-001', 'name': 'gpt-testopenai-dev-eastus-001', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'system_data': {'created_by': 'marlosbosso@outlook.com', 'created_by_type': 'User', 'created_at': '2023-03-11T20:34:16.493868Z', 'last_modified_by': 'marlosbosso@outlook.com', 'last_modified_by_type': 'User', 'last_modified_at': '2023-03-11T20:34:16.493868Z'}, 'etag': '"4a371f3a-a723-4a6d-9f25-473f1053f098"', 'properties': {'provisioning_state': 'Succeeded', 'model': {'format': 'OpenAI', 'name': 'gpt-35-turbo', 'version': '0301'}, 'scale_settings': {'scale_type': 'Standard'}, 'capabilities': {'completion': 'true', 'chatCompletion': 'true'}}}


### Call API

In [ ]:
import openai

In [ ]:
key = cognitive_client.accounts.list_keys(resource_group_name = resource_group_name,
                                                                 account_name = openai_name).key1

In [ ]:
openai.api_type = "azure"
openai.api_base = cognitive_account.properties.endpoint
openai.api_version = "2022-12-01"
openai.api_key = key

In [ ]:
response = openai.Completion.create(
  engine=deployment_name,
  prompt="<|im_start|>system\nAssistant is a large language model trained by OpenAI.\n<|im_end|>\n<|im_start|>user\nWhat's the difference between garbanzo beans and chickpeas?\n<|im_end|>\n<|im_start|>assistant\n",
  temperature=0,
  max_tokens=500,
  top_p=0.5,
  stop=["<|im_end|>"])

print(response['choices'][0]['text'])

### Delete resources

In [ ]:
deployment_deletion = cognitive_client.deployments.begin_delete(resource_group_name = resource_group_name,
                                                                 account_name = openai_name,
                                                                 deployment_name = deployment_name)

while deployment_deletion.status() == 'InProgress':
    print(deployment_deletion.status(), 'waiting 10 seconds....')
    time.sleep(10)

print(deployment_deletion.status())

In [7]:
account_deletion = cognitive_client.accounts.begin_delete(resource_group_name = resource_group_name,
                                                                 account_name = openai_name)

In [ ]:
resource_group_deletion = resource_client.resource_groups.begin_delete(resource_group_name)

while resource_group_deletion.status() == 'InProgress':
    print(resource_group_deletion.status(), 'waiting 10 seconds....')
    time.sleep(10)

print(resource_group_deletion.status())

In [ ]:
cognitive_client.accounts.